Source: https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsbyvaccinationstatusengland

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

In [3]:
rr = pd.read_csv(r"C:\Users\jdgoe\Downloads\referencetablefeb213.xlsx - Table 2 (1).csv")
rr.head()

,Cause of Death,Year,Month,Age group,Vaccination status,Count of deaths,Person-years,"Age-standardised mortality rate / 100,000 person-years",Noted as Unreliable,Lower confidence limit,Upper confidence limit
0,All causes,2021,April,18-39,Unvaccinated,283,"850,131",33.9,NaN,30.0,37.9
1,All causes,2021,April,18-39,"First dose, less than 21 days ago",18,"40,183",40.2,u,23.6,63.9
2,All causes,2021,April,18-39,"First dose, at least 21 days ago",194,"166,571",111.2,NaN,95.2,127.3
3,All causes,2021,April,18-39,"Second dose, less than 21 days ago",19,"42,438",42.8,u,25.5,67.1
4,All causes,2021,April,18-39,"Second dose, at least 21 days ago",15,"30,337",56.5,u,28.0,98.4


In [4]:
rr['Year'] = rr['Year'].astype(str)
rr['Age-standardised mortality rate / 100,000 person-years'] = pd.to_numeric(rr['Age-standardised mortality rate / 100,000 person-years'], errors='coerce')
rr = rr.rename(columns={'Age-standardised mortality rate / 100,000 person-years': 'rate'})

In [5]:
rtwo = rr[(rr['Year'] == '2022') & (rr['Cause of Death'] == 'All causes')]
rone = rr[(rr['Year'] == '2021') & (rr['Cause of Death'] == 'All causes')]

In [6]:
rt = rtwo

In [7]:
rt = rt.replace(['nan', 'x'], np.nan)
rt = rt.replace('x', np.nan)
rt = rt.fillna('0') # fill nan and x with 0

In [8]:
# turn mortality rate and confidence limits into integers
rt['rate'] = pd.to_numeric(rt['rate'], errors='coerce')
rt['rate'] = rt['rate'].astype(int)
rt['Upper confidence limit'] = pd.to_numeric(rt['Upper confidence limit'], errors='coerce')
rt['Upper confidence limit'] = rt['Upper confidence limit'].astype(int)
rt['Lower confidence limit'] = pd.to_numeric(rt['Lower confidence limit'], errors='coerce')
rt['Lower confidence limit'] = rt['Lower confidence limit'].astype(int)

In [9]:
rt['SD'] = (rt['Upper confidence limit'] - rt['Lower confidence limit']) / 2

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.lineplot(x="Month", y="rate", hue="Vaccination status", data=rt)

In [ ]:
fig = plt.figure(figsize=(16,14))
sns.scatterplot(x="Month", y="rate", hue="Vaccination status", data=rt)
plt.ylim(0,60000)

In [12]:
r1 = rt[rt['Month'].isin(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])]
r2 = rt[rt['Month'].isin(['September', 'October', 'November', 'December'])]

In [13]:
# remove insignificant data
r2 = r2[r2['Vaccination status'] != 'First dose, less than 21 days ago']
r2 = r2[r2['Vaccination status'] != 'Second dose, less than 21 days ago']
r2 = r2[r2['Vaccination status'] != 'Third dose or booster, less than 21 days ago']
r1 = r1[r1['Vaccination status'] != 'First dose, less than 21 days ago']
r1 = r1[r1['Vaccination status'] != 'Second dose, less than 21 days ago']
r1 = r1[r1['Vaccination status'] != 'Third dose or booster, less than 21 days ago']

In [14]:
ra = r2[r2['Age group'].isin(['18-39', '40-49', '50-59', '60-69', '70-79', '90+'])]
rb = r1[r1['Age group'].isin(['18-39', '40-49', '50-59', '60-69', '70-79', '90+'])]

In [32]:
# Get the unique age group values
age_group_values = rb['Age group'].unique()

# Create a figure with six subplots
fig, axs = plt.subplots(2, 3, figsize=(30, 20))
axs[0, 2].text(-0.5, 1.1, "Source: https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsbyvaccinationstatusengland", transform=axs[0, 2].transAxes,
               horizontalalignment='left', verticalalignment='top', size= 11)


# Loop over the age group values and create a scatter plot for each
for i, age_group in enumerate(age_group_values):
    # Subset the data for the current age group value
    age_group_data = rb.loc[rb['Age group'] == age_group]

    # Create a scatter plot for the current age group value
    row = i // 3
    col = i % 3
    sns.lineplot(x="Month", y="rate", hue="Vaccination status", data=age_group_data, ax=axs[row, col])
    axs[row, col].set_title(f"Age Group: {age_group}")
    axs[row, col].set_xlabel('')
    axs[row, col].set_ylabel('')
    axs[row, col].tick_params(axis='x', labelrotation=45)
    axs[row, col].grid(alpha=.5)
    if col == 0:

        axs[row, col].set_ylabel('Age-standardized mortality rate (per 100,000)', size=20)

fig.suptitle('Mortality Rate by C-19 Vaccination Status\n(U.K. 2022)', size=45)
plt.rcParams['figure.dpi'] = 360
# Show the plot
plt.show()